<a href="https://colab.research.google.com/github/Cyberoctane29/Language-Translation-System-Using-Neural-Networks/blob/main/NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Language Translation System using Neural Network
##Model: mT5

In [ ]:
!pip install transformers sentencepiece datasets

#the three libraries to be used
# Transformers: Library for working with transformer-based models, including model loading and configuration.
# Sentencepiece: Tokenization library for breaking down text into smaller units, useful for NLP tasks.
# Datasets: Library for managing and accessing datasets, facilitating data preparation for training and testing.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
# Importing necessary libraries for the project

from datasets import load_dataset  # For loading and managing datasets
from google.colab import drive  # For interacting with Google Drive
from IPython.display import display  # For displaying visual elements in the notebook
from IPython.html import widgets  # For interactive widgets in IPython
import matplotlib.pyplot as plt  # For creating visualizations and plots
import numpy as np  # For numerical operations
import seaborn as sns  # For enhancing visualizations
import torch  # PyTorch library for deep learning
from torch import optim  # PyTorch's optimization module
from torch.nn import functional as F  # PyTorch's functional module
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer  # For working with transformer-based models
from transformers import get_linear_schedule_with_warmup  # For learning rate scheduling
from tqdm import tqdm_notebook  # For displaying progress bars in Jupyter notebooks

sns.set()  # Set the default Seaborn theme for visualizations

/usr/local/lib/python3.10/dist-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


In [ ]:
# Our model (Hugging Face model repository) is pre-trained on a bunch of unlabeled text,
# but we will fine-tune and improve it for our specific goal, which is machine translation.
# We choose 'google/mt5-small' as the base model; 'mt5-base' requires the pro version of Colab, so we use 'mt5-small'.

model_repo = 'google/mt5-small'
max_seq_len = 20  # Set the maximum sequence length for tokenization

In [ ]:
#loading the tokenizer into the model

tokenizer=AutoTokenizer.from_pretrained(model_repo)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Model description: https://huggingface.co/google/mt5-small

model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cpu()  # We put it onto our CPU; additionally, we may use CUDA to put it on our GPU
# We will use the particular languages from the model whose datasets are properly curated

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
input_sent='We are group-157'
token_ids=tokenizer.encode(input_sent,return_tensors='pt').cpu() #encode is the function which will tokenize
#we used pt=pyTorch, tensorflow could have been used as well
token_ids

# Generating output sequence from the model based on the input token IDs

model_out = model.generate(token_ids)
print(model_out)

# Converting the output token IDs to a human-readable string

output_text = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(model_out[0]))
print(output_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[     0, 250099,      1]])
<pad> <extra_id_0></s>


In [ ]:
# Example input string with a language tag

example_input_str = '<jp>This is a test nbuig.'

# Encoding the input string and getting the input IDs

input_ids = tokenizer.encode(example_input_str, return_tensors='pt')
print('Input IDs:', input_ids)

# Converting the input IDs back to tokens

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
print('Tokens:', tokens)

# Underscore basically implies that there is a space or it's the start of a character,
# but it differs from tokenizer to tokenizer. As 'nbuig' is not an existing word in the vocab,
# it got split up into tokens.

Input IDs: tensor([[ 1042,  3889,   669, 13673,   339,   259,   262,  2978,   259,   272,
          1982,  1315,   260,     1]])
Tokens: ['▁<', 'jp', '>', 'This', '▁is', '▁', 'a', '▁test', '▁', 'n', 'bu', 'ig', '.', '</s>']


In [ ]:
# Sorting the tokenizer's vocabulary items based on their indices
#sorted(tokenizer.vocab.items(), key=lambda x: x[1])

In [ ]:
# Loading the 'alt' dataset using the Hugging Face Datasets library

# Source: https://huggingface.co/datasets/alt
dataset = load_dataset('alt')

Generating train split:   0%|          | 0/18088 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1019 [00:00<?, ? examples/s]

In [ ]:
# Separating the 'train' and 'test' datasets from the loaded dataset

train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# Displaying the first entry in the 'train_dataset'

train_dataset[0]

{'SNT.URLID': '80188',
 'SNT.URLID.SNTID': '1',
 'url': 'http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal',
 'translation': {'bg': 'ফ্রান্সের প্যারিসের পার্ক দি প্রিন্সেস-এ হওয়া ২০০৭-এর রাগবি বিশ্বকাপের পুল সি-তে ইটালি পর্তুগালকে ৩১-৫ গোলে হারিয়েছে।',
  'en': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes, Paris, France.',
  'en_tok': 'Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes , Paris , France .',
  'fil': 'Natalo ng Italya ang Portugal sa puntos na 31-5 sa Grupong C noong 2007 sa Pandaigdigang laro ng Ragbi sa Parc des Princes, Paris, France.',
  'hi': '2007 में फ़्रांस, पेरिस के पार्क डेस प्रिंसेस में हुए रग्बी विश्व कप के पूल C में इटली ने पुर्तगाल को 31-5 से हराया।',
  'id': 'Italia berhasil mengalahkan Portugal 31-5 di grup C dalam Piala Dunia Rugby 2007 di Parc des Princes, Paris, Perancis.',
  'ja': 'フランスのパリ、パルク・デ・プランスで行われた2007年ラグビーワールドカップのプールCで、イタリアは31対5でポルトガルを

In [ ]:
# Mapping language codes to special tokens

LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'ja': '<jp>',
    'zh': '<zh>'
}

In [ ]:
# Adding additional special tokens to the tokenizer

special_tokens_dict = {'additional_special_tokens': list(LANG_TOKEN_MAPPING.values())}
tokenizer.add_special_tokens(special_tokens_dict)

# Resizing token embeddings in the model based on the updated tokenizer

model.resize_token_embeddings(len(tokenizer))

Embedding(250103, 512)

In [ ]:
# Encoding the example input string with padding, truncation, and setting max length

token_ids = tokenizer.encode(
    example_input_str, return_tensors='pt', padding='max_length',
    truncation=True, max_length=max_seq_len)
print(token_ids)

tensor([[250102,   1494,    339,    259,    262,   2978,    259,    272,   1982,
           1315,    260,      1,      0,      0,      0,      0,      0,      0,
              0,      0]])


In [ ]:
# Function to encode the input string

def encode_input_str(text, target_lang, tokenizer, seq_len, lang_token_map=LANG_TOKEN_MAPPING):
    target_lang_token = lang_token_map[target_lang]

    # Tokenize and add special tokens

    input_ids = tokenizer.encode(
        text=target_lang_token + text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=seq_len)

    return input_ids[0]

# Function to encode the target string

def encode_target_str(text, tokenizer, seq_len, lang_token_map=LANG_TOKEN_MAPPING):
    token_ids = tokenizer.encode(
        text=text,
        return_tensors='pt',
        padding='max_length',
        truncation=True,
        max_length=seq_len)

    return token_ids[0]

# Function to format translation data

def format_translation_data(translations, lang_token_map, tokenizer, seq_len=128):
    # Choose a random 2 languages for input and output

    langs = list(lang_token_map.keys())
    input_lang, target_lang = np.random.choice(langs, size=2, replace=False)

    # Get the translations for the batch

    input_text = translations[input_lang]
    target_text = translations[target_lang]

    if input_text is None or target_text is None:
        return None

    input_token_ids = encode_input_str(
        input_text, target_lang, tokenizer, seq_len, lang_token_map)

    target_token_ids = encode_target_str(
        target_text, tokenizer, seq_len, lang_token_map)

    return input_token_ids, target_token_ids

# Function to transform a batch of data

def transform_batch(batch, lang_token_map, tokenizer):
    inputs = []
    targets = []
    for translation_set in batch['translation']:
        formatted_data = format_translation_data(
            translation_set, lang_token_map, tokenizer, max_seq_len)

        if formatted_data is None:
            continue

        input_ids, target_ids = formatted_data
        inputs.append(input_ids.unsqueeze(0))
        targets.append(target_ids.unsqueeze(0))

    batch_input_ids = torch.cat(inputs).cpu()
    batch_target_ids = torch.cat(targets).cpu()

    return batch_input_ids, batch_target_ids

# Function to get a data generator

def get_data_generator(dataset, lang_token_map, tokenizer, batch_size=32):
    dataset = dataset.shuffle()
    for i in range(0, len(dataset), batch_size):
        raw_batch = dataset[i:i+batch_size]
        yield transform_batch(raw_batch, lang_token_map, tokenizer)


In [ ]:
# Example of using the functions

in_ids, out_ids = format_translation_data(train_dataset[1]['translation'], LANG_TOKEN_MAPPING, tokenizer)
print('Input Text:', ''.join(tokenizer.convert_ids_to_tokens(in_ids)))
print('Output Text:', ''.join(tokenizer.convert_ids_to_tokens(out_ids)))

# Example of using the data generator

data_gen = get_data_generator(train_dataset, LANG_TOKEN_MAPPING, tokenizer, 8)
data_batch = next(data_gen)
print('Input shape:', data_batch[0].shape)
print('Output shape:', data_batch[1].shape)

Input Text: <zh>▁Andrea▁Masi▁opened▁the▁scoring▁in▁the▁fourth▁minute▁with▁a▁try▁for▁Italy.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Output Text: ▁安德里亚·马西在第四分钟成功达阵,意大利率先得分。</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [ ]:
# Constants for training

n_epochs = 8  # Number of training epochs
batch_size = 16  # Batch size
print_freq = 50  # Frequency of printing training information
checkpoint_freq = 1000  # Frequency of saving model checkpoints
lr = 5e-4  # Learning rate
n_batches = int(np.ceil(len(train_dataset) / batch_size))  # Total number of batches
total_steps = n_epochs * n_batches  # Total training steps
n_warmup_steps = int(total_steps * 0.01)  # Number of warm-up steps for learning rate

In [ ]:
# Optimizer setup

optimizer = AdamW(model.parameters(), lr=lr)  # AdamW optimizer with specified learning rate
scheduler = get_linear_schedule_with_warmup(
    optimizer, n_warmup_steps, total_steps)  # Linear scheduler with warm-up for learning rate

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []  # List to store training losses

In [ ]:
def eval_model(model, gdataset, max_iters=8):

    # Evaluate the model on the given dataset.

    # Parameters:
    # - model: The trained model to evaluate.
    # - gdataset: The dataset for evaluation.
    # - max_iters: Maximum number of iterations for evaluation.

    # Returns:
    # - Mean evaluation loss.

    test_generator = get_data_generator(gdataset, LANG_TOKEN_MAPPING, tokenizer, batch_size)
    eval_losses = []
    for i, (input_batch, label_batch) in enumerate(test_generator):
        if i >= max_iters:
            break

        model_out = model.forward(
            input_ids=input_batch,
            labels=label_batch
        )
        eval_losses.append(model_out.loss.item())

    return np.mean(eval_losses)

In [ ]:
# Evaluate the model on the test dataset and calculate the average loss

test_loss = eval_model(model, test_dataset)

In [ ]:
# Displaying the test loss

test_loss

25.2537579536438

In [ ]:
# Training loop

for epoch_idx in range(n_epochs):

  # Randomize data order

  data_generator = get_data_generator(train_dataset, LANG_TOKEN_MAPPING,
                                      tokenizer, batch_size)

  for batch_idx, (input_batch, label_batch) \
      in tqdm_notebook(enumerate(data_generator), total=n_batches):

    optimizer.zero_grad()

    # Forward pass

    model_out = model.forward(
        input_ids = input_batch,
        labels = label_batch)

    # Calculate loss and update weights

    loss = model_out.loss
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Print training update info

    if (batch_idx + 1) % print_freq == 0:
      avg_loss = np.mean(losses[-print_freq:])
      print('Epoch: {} | Step: {} | Avg. loss: {:.3f} | lr: {}'.format(
          epoch_idx+1, batch_idx+1, avg_loss, scheduler.get_last_lr()[0]))

# Evaluate the model on the test dataset

test_loss = eval_model(model, test_dataset)
print('Test loss of {:.3f}'.format(test_loss))

In [ ]:
# Graph the loss

window_size = 50
smoothed_losses = []

# Calculate moving average for better visualization

for i in range(len(losses) - window_size):
  smoothed_losses.append(np.mean(losses[i:i + window_size]))

plt.plot(smoothed_losses[100:])
plt.title('Smoothed Training Loss Over Time')
plt.xlabel('Training Steps')
plt.ylabel('Smoothed Loss')
plt.show()

In [ ]:
# @title MANUAL TESTING

test_sentence = test_dataset[0]['translation']['en']
print('Raw input text:', test_sentence)

# Encode input text for Japanese translation

input_ids = encode_input_str(
    text=test_sentence,
    target_lang='ja',
    tokenizer=tokenizer,
    seq_len=model.config.max_length,
    lang_token_map=LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cpu()

# Display truncated input text

print('Truncated input text:', tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[0])))

In [ ]:
output_tokens = model.generate(input_ids, num_beams=10, num_return_sequences=3)

# Decode and print the generated output sequences

for token_set in output_tokens:
    decoded_output = tokenizer.decode(token_set, skip_special_tokens=True)
    print(decoded_output)

In [ ]:
# @title Translator

input_text = 'The project is a very good project.' #@param {type:"string"}
output_language = 'ja' #@param ["en", "ja", "zh"]

# Encode the input text and generate the translation

input_ids = encode_input_str(
    text=input_text,
    target_lang=output_language,
    tokenizer=tokenizer,
    seq_len=model.config.max_length,
    lang_token_map=LANG_TOKEN_MAPPING)
input_ids = input_ids.unsqueeze(0).cpu()

output_tokens = model.generate(input_ids, num_beams=20, length_penalty=0.2)

# Decode and print the generated translation

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print(input_text + '  ->  ' + output_text)